# Исследовательская работа по теме "Выделение структурных признаков символов с помощью клеточных автоматов"

## Шаг 1. 
На вход последовательности клеточных автоматов поступает изображение символа. 

In [1]:
from glob import glob
from skimage import io
import matplotlib.pyplot as plt

In [2]:
size = 28

### Предварительная обработка изображения.
В данной работе система распознований использует в своей работе клетки, имеющие два состояния, соответствующие к белому и черному цвету пикселей изображения.
Программа работает только с одноканальными изображениями, поэтому, прежде чем приступить, необходимо преобразовать RGB-изображения.

In [3]:
def is_colored(image):
    if len(image.shape) == 2:
        return False
    else:
        return True

Для выполнения этой задачи может быть выбран клеточный автомат, который будет определять каждую клетку соответствующей точке
на изображении, а локальный радиус клеток равен нулю. Автомат будет выполнять одно из трех действий: преобразовывать цвет каждой точки входного изображения в оттенок серого; закрашивать клетку черным цветом, если клетка оказывается темнее определенного предельного цвета; окрашивать клетку белым цветом, если клетка имеет цвет светлее определьного цвета.

In [4]:
def rgb2gray(pix):
    for i in range(size):
        for j in range(size):
            a = pix[i,j][0]
            b = pix[i,j][1]
            c = pix[i,j][2]
            S = 0.3 * a + 0.59 * b + 0.11 * c
            if (S > 128):
                pix[i,j][0] = 255
                pix[i,j][1] = 255
                pix[i,j][2] = 255
            else:
                pix[i,j][0] = 0
                pix[i,j][1] = 0
                pix[i,j][2] = 0
    return np.round(np.sum(pix, axis=2)).astype(np.uint8).reshape((size, size))

In [5]:
def mono(pix):
    for i in range(size):
        for j in range(size):
            if (pix[i,j] > 128):
                pix[i,j] = 255
            else:
                pix[i,j] = 0
    return pix

## Шаг 1

Создадим массив, в котором будем хранить метки автомата: красный - 1, серый - 2, синий - 3, зеленый - 4, оранжевый - 5 и т.д.. Второй массив нам требуется, т.к. все действия с клетками совершаются одновременно. Каждый элемент array[i][j] содержит четыре элемента, т.к. это предельно возможное количество меток в автомате.

In [6]:
red = 1
grey = 2
blue = 3
green = 4
orange = 5
azure = 6
brown = 7
yellow = 8
purple = 9
maroon = 10

In [7]:
import numpy as np
array = np.zeros(shape=(size,size,7))
array1 = np.copy(array)

Также нам потребуются три вспомогательных массива для проверки на шагах 3, 5, 11 и 22. Обозначим их narr1, narr2 и narr3.

## Приступим ко второму шагу автомата: 
найдем верхнюю левую белую клетку и поставим нужные метки. Верхняя левая черная клетка изображения помечается красной и серой метками. 

In [8]:
def step2(img):
    global array,array1, i,j
    flag = 0
    while (i < size) and not flag:
        while (j < size) and not flag:
            if img[i][j] == 255:
                flag = 1 
            else:
                j+=1
        if flag == 0:
            i+=1
            j = 0

    array[i][j][0] = red
    array[i][j][1] = grey

## Шаг 3. Автомат «создание шлейфа»: 
красная метка заменяется синей меткой. 

In [9]:
def step3():
    global array, array1, red, blue, i, size
    flag = 0
    line = i
    column = 0
    while (line < size) and not flag:
        while (column < size) and not flag:
            if array[line][column][0] == red:
                array[line][column][0] = blue
                flag = 1
            else:
                column+=1
        column = 0
        line+=1
    array1 = np.copy(array)

## Шаг 4. Автомат «создание фронта»: 
белые клетки без серой метки рядом с клетками с серой меткой помечаются серой и красной метками. 

In [10]:
def trycolour8(x,y,z,n):
    global array
    if x == 0:
        if y == 0:
            if (array[x + 1][y][n] == z) or (array[x + 1][y + 1][n] == z) or (array[x][y + 1][n] == z):
                return 1
        elif y == (size - 1):
            if (array[x + 1][y][n] == z) or (array[x + 1][y - 1][n] == z) or (array[x][y - 1][n] == z):
                return 1
        else:
            if (array[x + 1][y][n] == z) or (array[x + 1][y + 1][n] == z) or (array[x][y + 1][n] == z) or (array[x][y - 1][n] == z) or (array[x + 1][y - 1][n] == z):
                return 1
    elif x == (size - 1):
        if y == 0:
            if (array[x - 1][y][n] == z) or (array[x - 1][y + 1][n] == z) or (array[x][y + 1][n] == z):
                return 1
        elif y == (size - 1):
            if (array[x - 1][y][n] == z) or (array[x - 1][y - 1][n] == z) or (array[x][y - 1][n] == z):
                return 1
        else:
            if (array[x - 1][y][n] == z) or (array[x - 1][y + 1][n] == z) or (array[x][y + 1][n] == z) or (array[x][y - 1][n] == z) or (array[x - 1][y - 1][n] == z):
                return 1
    elif y == 0:
        if (array[x - 1][y][n] == z) or (array[x - 1][y + 1][n] == z) or (array[x][y + 1][n] == z) or (array[x + 1][y + 1][n] == z) or (array[x + 1][y][n] == z):
            return 1
    elif y == (size - 1):
        if (array[x - 1][y][n] == z) or (array[x - 1][y - 1][n] == z) or (array[x][y - 1][n] == z) or (array[x + 1][y - 1][n] == z) or (array[x + 1][y][n] == z):
            return 1
    else:
        if (array[x - 1][y][n] == z) or (array[x - 1][y - 1][n] == z) or (array[x][y - 1][n] == z) or (array[x + 1][y - 1][n] == z) or (array[x + 1][y][n] == z) or (array[x + 1][y + 1][n] == z) or (array[x][y + 1][n] == z)  or (array[x - 1][y + 1][n] == z):
            return 1
    return 0

In [11]:
def trycolour4(x,y,z,n):
    global array
    if x == 0:
        if y == 0:
            if (array[x + 1][y][n] == z) or (array[x][y + 1][n] == z):
                return 1
        elif y == (size - 1):
            if (array[x + 1][y][n] == z) or (array[x][y - 1][n] == z):
                return 1
        else:
            if (array[x + 1][y][n] == z) or (array[x][y + 1][n] == z) or (array[x][y - 1][n] == z):
                return 1
    elif x == (size - 1):
        if y == 0:
            if (array[x - 1][y][n] == z) or (array[x][y + 1][n] == z):
                return 1
        elif y == (size - 1):
            if (array[x - 1][y][n] == z) or (array[x][y - 1][n] == z):
                return 1
        else:
            if (array[x - 1][y][n] == z) or (array[x][y + 1][n] == z) or (array[x][y - 1][n] == z):
                return 1
    elif y == 0:
        if (array[x - 1][y][n] == z) or (array[x][y + 1][n] == z) or (array[x + 1][y][n] == z):
            return 1
    elif y == (size - 1):
        if (array[x - 1][y][n] == z) or (array[x][y - 1][n] == z) or (array[x + 1][y][n] == z):
            return 1
    else:
        if (array[x - 1][y][n] == z) or (array[x][y - 1][n] == z) or (array[x + 1][y][n] == z) or (array[x][y + 1][n] == z):
            return 1
    return 0

In [12]:
def step4(img):
    global array, array1,red, grey, size
    for line in range(size):
        for column in range(size):
            if (img[line, column] == 255) and (array[line][column][1] != grey):
                if (trycolour4(line, column, grey, 1) == 1):
                    array1[line][column][0] = red
                    array1[line][column][1] = grey
    array = np.copy(array1)

## Шаг 5. 
Одна клетка рядом с красной меткой заменяет синюю метку зеленой. 

In [13]:
def step5():
    global array,red, grey, green, i, size
    flag = 0
    line = i
    column = 0
    while (line < size) and not flag:
        while (column < size) and not flag:
            if (trycolour8(line,column,red,0) == 1) and (array[line][column][0] == blue):
                array[line][column][0] = green
                flag = 1
            else:
                column+=1
        column = 0
        line+=1

## Шаг 6. Автомат «проход по шлейфу»: 
все клетки рядом с клетками с зелеными метками заменяют синие метки зелеными. 

In [14]:
def step6():
    global array, array1,red, grey, green, i, blue, size
    array1 = np.copy(array)
    for line in range (i, size):
        for column in range(size):
            if ((trycolour8(line,column,green,0) == 1) or (trycolour8(line,column,green,2) == 1)) and (array[line][column][0] == blue):
                array1[line][column][0] = green
    array = np.copy(array1)

## Шаг 7. Автомат «проход по фронту»: 
все клетки с красной, но без зеленой метки, находящиеся рядом с клетками с зеленой меткой, помечаются зеленой меткой; если клетка с красной и зеленой меткой находится рядом с клеткой с синей меткой, то данная клетка помечается оранжевой меткой. 

In [15]:
def step7():
    global array, array1,red, grey, green, i, orange, size
    for line in range (i, size):
        for column in range(size):
            if (array[line][column][0] == red):
                if (array[line][column][2] != green):
                    if (trycolour8(line,column,green,0) == 1) or (trycolour8(line,column,green,2) == 1):
                        array1[line][column][2] = green
                elif (trycolour8(line,column,blue,0) == 1):
                    array1[line][column][3] = orange
    array = np.copy(array1)

## Шаг 8. 
Если, начиная с шага 5, автоматы не изменили состояния ни одной клетки, то перейти на шаг 9, иначе перейти на шаг 5. 

## Шаг 9. Автомат «удаление зеленых меток»: 
все зеленые метки у клеток удаляются. 

In [16]:
def step9():
    global array, array1,green, i, size
    for line in range (i, size):
        for column in range(size):
            if array[line][column][0] == green:
                array[line][column][0] = 0
            if array[line][column][2] == green:
                array[line][column][2] = 0
    array1 = np.copy(array)

## Шаг 10.  
Если у одной из клеток присутствует синяя метка, то перейти на шаг 11, иначе на шаг 20.  

## Шаг 11.  Автомат «создание эха»: 
синюю метку клеток заменить коричневой и голубой, к красной метке символов добавить коричневую метку. 

In [17]:
def step11():
    global array,brown, i, size, blue, azure
    for line in range (i, size):
        for column in range(size):
            if array[line][column][0] == blue:
                array[line][column][4] = brown
                array[line][column][0] = azure
            elif array[line][column][0] == red:
                array[line][column][4] = brown

## Шаг 12.  Автомат «создание шлейфа эха»: 
коричневая метка заменяется желтой меткой. 

In [18]:
def step12():
    global array, array1,green, i, size, blue, azure, yellow
    for line in range (i, size):
        for column in range(size):
            if array[line][column][4] == brown:
                array[line][column][4] = yellow
    array1 = np.copy(array)

## Шаг 13.  Автомат «создание фронта эха»: 
белые клетки с серой меткой, но без фиолетовой метки рядом с клетками с фиолетовой меткой помечаются фиолетовой и коричневой метками. 

In [19]:
def step13(img):
    global red, grey, size,array, array1
    for line in range(size):
        for column in range(size):
            if (img[line, column] == 255) and (array[line][column][1] == grey):
                if (array[line][column][5] != purple) and (trycolour8(line,column,purple,5) == 1):
                    array1[line][column][5] = purple
                    array1[line][column][4] = brown
    array = np.copy(array1)

## Шаг 14.  
Одна клетка рядом с коричневой меткой меняет желтую метку малиновой. 

In [20]:
def step14():
    global array,brown, yellow, maroon, i, size
    flag = 0
    line = i
    column = 0
    while (line < size) and not flag:
        while (column < size) and not flag:
            if (trycolour8(line,column,brown,4) == 1) and (array[line][column][4] == yellow):
                array[line][column][4] = maroon
                flag = 1
            else:
                column+=1
        column = 0
        line+=1

## Шаг 15.  Автомат «проход по шлейфу эха»: 
все клетки рядом с клетками с малиновыми метками заменяют желтые метки малиновыми. 

In [21]:
def step15():
    global array, array1,maroon, yellow, size
    for line in range(size):
        for column in range(size):
            if (array[line][column][4] == yellow) and (trycolour8(line,column,maroon,4) == 1):
                array1[line][column][4] = maroon
    array = np.copy(array1)

## Шаг 16.  Автомат «проход по фронту эха»: 
все клетки с коричневой, но не с малиновой меткой, находящиеся рядом с клетками с малиновой меткой, помечаются малиновой меткой; если клетка с коричневой и малиновой метками находится рядом с клеткой с желтой меткой, то данная клетка помечается пурпурной меткой. 

In [22]:
def step16():
    global array, array1,maroon, brown, size
    for line in range(size):
        for column in range(size):
            if (array[line][column][4] == brown) and (array[line][column][6] != maroon) and ((trycolour8(line,column,maroon,4) == 1) or (trycolour8(line,column,maroon,6) == 1)):
                array1[line][column][6] = maroon
            elif (array[line][column][4] == brown) and (array[line][column][6] == maroon) and (trycolour8(line,column,yellow,4) == 1):
                array1[line][column][5] = purple
    array = np.copy(array1)

## Шаг 17.  
Если, начиная с шага 14, автоматы не изменили состояния ни одной клетки, то перейти на шаг 18, иначе перейти на шаг 14. 

## Шаг 18.  Автомат «удаление малиновых меток»: 
все малиновые метки у клеток убираются. 

In [23]:
def step18():
    global array, array1,maroon, i, size
    for line in range (i, size):
        for column in range(size):
            if array[line][column][4] == maroon:
                array[line][column][4] = 0
            if array[line][column][6] == maroon:
                array[line][column][6] = 0
    array1 = np.copy(array)

## Шаг 19.  
Если, начиная с шага 12, автоматы не изменили состояния ни одной  клетки, то перейти на шаг 20, иначе перейти на шаг 12. 

## Шаг 20.  
Если, начиная с шага 3, автоматы не изменили состояния ни одной  клетки, то перейти на шаг 21, иначе перейти на шаг 3. 

## Шаг 21.  
Верхняя левая черная клетка изображения помечается синей меткой.

In [24]:
def step21():
    global i,j,array
    array[i][j][0] = blue

## Шаг 22.  
Автомат «удание нижних лишних меток»: если над клеткой с синей или оранжевой меткой находится клетка с такой же меткой, то в данной клетке эту метку удалить. 

In [25]:
def checkup(x,y,z):
    global blue, orange,array
    if x == 0:
        return 0
    elif z == blue:
        if array[x - 1][y][0] == z:
            return 1
        else:
            return 0
    elif z == orange:
        if array[x - 1][y][3] == z:
            return 1
        else:
            return 0

In [26]:
def step22():
    global array, array1,orange, i, blue, size
    for line in range (i, size):
        for column in range(size):
            if (array[line][column][0] == blue) and (checkup(line,column,blue) == 1):
                array1[line][column][0] = 0
            if (array[line][column][3] == orange) and (checkup(line,column,orange) == 1):
                array1[line][column][3] = 0
    array = np.copy(array1)

## Шаг 23.  Автомат «удаление лишних меток справа»: 
если слева от клетки с синей или оранжевой меткой находится клетка с такой же меткой, то в данной клетке эту метку удалить. 

In [27]:
def checkleft(x,y,z):
    global blue, orange,array
    if y == 0:
        return 0
    elif z == blue:
        if array[x][y - 1][0] == z:
            return 1
        else:
            return 0
    elif z == orange:
        if array[x][y - 1][3] == z:
            return 1
        else:
            return 0

In [28]:
def step23():
    global array, array1,i, blue, orange, size
    for line in range (i, size):
        for column in range(size):
            if (array[line][column][0] == blue) and (checkleft(line,column,blue) == 1):
                array1[line][column][0] = 0
            if (array[line][column][3] == orange) and (checkleft(line,column,orange) == 1):
                array1[line][column][3] = 0
    array = np.copy(array1)

## Шаг 24. 
Если, начиная с шага 12, автоматы не изменили состояния ни одной  клетки, то завершить последовательность, иначе перейти на шаг 12. 

### Проверка, есть ли синяя метка.

In [29]:
def isblue():
    global blue, i, size,array
    line = i
    column = 0
    while (line < size):
        while (column < size):
            if (array[line][column][0] == blue):
                return 1
            else:
                column+=1
        column = 0
        line+=1
    return 0

## Общий вид программы
Теперь необходимо собрать все шаги вместе, учтя проверки на шагах 3, 5, 11 и 22:

In [35]:
num = 0
features = []

for filename in glob(r'C:\Users\User\mmp\training_letters\*.png'):
    img = io.imread(filename)
    
    if is_colored(img):
        img = rgb2gray(img)
    img = mono(img)
    
    print(filename)  
    i = 0
    j = 0        
    step2(img)
    equal1 = 0
    while (not equal1):
        narr1 = np.copy(array)
        step3()
        step4(img)
        equal2 = 0
        while (not equal2):
            narr2 = np.copy(array)
            step5()
            step6()
            step7()
            equal2 = np.equal(narr2, array).all()
        step9()
        if (isblue() == 1):
            equal3 = 0
            while (not equal3):
                narr3 = np.copy(array)
                equal2 = 0
                while (not equal2):
                    narr2 = np.copy(array)
                    step11()
                    step12()
                    step13(img)
                    step14()
                    step15()
                    step16()
                    equal2 = np.equal(narr2, array).all()
                step18()
                equal3 = np.equal(narr3, array).all()      
        equal1 = np.equal(narr1, array).all()
    step21()
    equal1 = 0

    while (not equal1):
        narr1 = np.copy(array)
        step22()
        step23()
        equal1 = np.equal(narr1, array).all()

    for line in range (size):
        for column in range(size):
            if array[line][column][0] == azure:
                a = [num, line, column]
                features.append(a)

    array = np.zeros(shape=(size,size,7))
    array1 = np.copy(array)
    num += 1

C:\Users\User\mmp\training_letters\58b12fdfb6869.DvgNT.png
C:\Users\User\mmp\training_letters\58b12fe3afd4c.V6019.png
C:\Users\User\mmp\training_letters\58b12fe713694.4AALu.png
C:\Users\User\mmp\training_letters\58b12feb8d870.hBgnb.png
C:\Users\User\mmp\training_letters\58b12ff790b6c.fStSj.png
C:\Users\User\mmp\training_letters\58b12ffb1e122.yrRqK.png
C:\Users\User\mmp\training_letters\58b12ffe95549.7WrO1.png
C:\Users\User\mmp\training_letters\58b130078eb18.Ntce4.png
C:\Users\User\mmp\training_letters\58b1300c0b92c.oL4lC.png


KeyboardInterrupt: 

На выходе алгоритма некоторые клетки изображения помечены голубыми метками. Их мы и будем использовать для распознавания. Выделенные признаки заносим в csv таблицу:

In [34]:
import csv

global features
FILENAME = "features1.csv"

with open(FILENAME, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(features)

Ура, мы получили табличку с нужными нам данными. Можем работать дальше.